In [5]:
import requests
import pandas as pd
import json
from dateutil import parser

In [6]:
api_key = "84b3e57f239ee216d8098e4743d65d98-cd7f58d6713874c1d54c0cb780e860ba"
account_id = "101-001-26920175-001"
oanda_url_practice = "https://api-fxpractice.oanda.com/v3"

In [7]:
session = requests.Session()
session.headers.update({
    "Authorization" : f"Bearer {api_key}",
    "Content-Type" : "application/json"
})

In [8]:
params = dict(
    count = 10,
    granularity = "H1",
    price = "MBA"
)

In [9]:
url = f"{oanda_url_practice}/accounts/{account_id}/instruments"

In [10]:
response = session.get(url, params = None, headers = None, data = None)

In [11]:
data = response.json()

In [12]:
instrument_list = data['instruments']

In [13]:
len(instrument_list)

68

In [14]:
instrument_list[0].keys()


dict_keys(['name', 'type', 'displayName', 'pipLocation', 'displayPrecision', 'tradeUnitsPrecision', 'minimumTradeSize', 'maximumTrailingStopDistance', 'minimumTrailingStopDistance', 'maximumPositionSize', 'maximumOrderUnits', 'marginRate', 'guaranteedStopLossOrderMode', 'tags', 'financing'])

In [15]:
instrument_keys = ['name', 'type', 'displayName', 'pipLocation', 'displayPrecision', 'tradeUnitsPrecision', 'marginRate']

In [16]:
instrument_dict = {}
for item in instrument_list:
    key = item["name"]
    instrument_dict[key] = {k : item[k] for k in instrument_keys }

In [17]:
# Save/write the data in intruments.json file

with open("../data/instruments.json", "w") as f:
    f.write(json.dumps(instrument_dict, indent=2))

In [47]:
# Function to fetch candle data

def fetch_candles(pair_name, count=10, granularity="H1"):
    url = f"{oanda_url_practice}/instruments/{pair_name}/candles"
    params = dict(
        count = count,
        granularity = granularity,
        price = "MBA"
    )
    response = session.get(url, params = params, headers = None, data = None)
    data = response.json()

    if response.status_code == 200:
        if 'candles' not in data:
            data = []
        else:
            data = data['candles']
    return response.status_code, data


def get_candle_df(data):
    if len(data) == 0:
        return pd.DataFrame()
    else:
        #  extracting some final data
        # !TODO: Remove the nested for loop
        prices = ['mid', 'bid', 'ask']
        ohlc = ['o', 'h', 'l', 'c']

        final_data = []
        for candle in data:
            if candle['complete'] == False:
                continue
            else:
                new_dict = {}
                new_dict['time'] = parser.parse(candle['time'])
                new_dict['volume'] = candle['volume']
                for p in prices:
                    for x in ohlc:
                        new_dict[f"{p}_{x}"] = candle[p][x]
                        # print(f"{p}_{x}")
                final_data.append(new_dict)
        return pd.DataFrame.from_dict(final_data)
    
def create_data_file(pair_name, count=10, granularity="H1"):
        code, data = fetch_candles("EUR_USD", count, granularity)
        if code != 200:
            print("Failed", pair_name, data)
        if len(data) == 0:
            print("No candle", pair_name)
        candle_df = get_candle_df(data)
        candle_df.to_pickle(f"../data/{pair_name}_{granularity}.pkl")
        print(f"{pair_name} {granularity} {candle_df.shape[0]} candles, {candle_df.time.min()} {candle_df.time.max()}")

In [54]:
tradable_curr = ["EUR", "USD", "GBP", "JPY", "CHF", "NZD", "CAD", "AUD"]
for pair_1 in tradable_curr:
    for pair_2 in tradable_curr:
        final_pair = f"{pair_1}_{pair_2}"
        if final_pair in instrument_dict:
            for granu in ["H1", "H4"]:
                create_data_file(final_pair, count=4001, granularity=granu)

EUR_USD H1 4001 candles, 2023-02-02 05:00:00+00:00 2023-09-22 20:00:00+00:00
EUR_USD H4 4001 candles, 2021-03-03 02:00:00+00:00 2023-09-22 17:00:00+00:00
EUR_GBP H1 4001 candles, 2023-02-02 05:00:00+00:00 2023-09-22 20:00:00+00:00
EUR_GBP H4 4001 candles, 2021-03-03 02:00:00+00:00 2023-09-22 17:00:00+00:00
EUR_JPY H1 4001 candles, 2023-02-02 05:00:00+00:00 2023-09-22 20:00:00+00:00
EUR_JPY H4 4001 candles, 2021-03-03 02:00:00+00:00 2023-09-22 17:00:00+00:00
EUR_CHF H1 4001 candles, 2023-02-02 05:00:00+00:00 2023-09-22 20:00:00+00:00
EUR_CHF H4 4001 candles, 2021-03-03 02:00:00+00:00 2023-09-22 17:00:00+00:00
EUR_NZD H1 4001 candles, 2023-02-02 05:00:00+00:00 2023-09-22 20:00:00+00:00
EUR_NZD H4 4001 candles, 2021-03-03 02:00:00+00:00 2023-09-22 17:00:00+00:00
EUR_CAD H1 4001 candles, 2023-02-02 05:00:00+00:00 2023-09-22 20:00:00+00:00
EUR_CAD H4 4001 candles, 2021-03-03 02:00:00+00:00 2023-09-22 17:00:00+00:00
EUR_AUD H1 4001 candles, 2023-02-02 05:00:00+00:00 2023-09-22 20:00:00+00:00